<a href="https://colab.research.google.com/github/poonampal/AIML/blob/main/gradient_discent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://github.com/codebasics/deep-learning-keras-tf-tutorial/blob/master/6_gradient_descent/insurance_data.csv

--2023-05-25 03:56:25--  https://github.com/codebasics/deep-learning-keras-tf-tutorial/blob/master/6_gradient_descent/insurance_data.csv
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘insurance_data.csv’

insurance_data.csv      [ <=>                ] 151.54K  --.-KB/s    in 0.08s   

2023-05-25 03:56:26 (1.90 MB/s) - ‘insurance_data.csv’ saved [155181]



In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('/content/insurance_data.csv', on_bad_lines='skip')
df.head()

,age,affordibility,bought_insurance
0,22,1,0
1,25,0,0
2,47,1,1
3,52,0,0
4,46,1,1


In [3]:
df.isnull().sum()

age                 0
affordibility       0
bought_insurance    0
dtype: int64

In [4]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Model

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
X_train, X_test,y_train, y_test = train_test_split(df[['age','affordibility']], df['bought_insurance'], test_size=0.2, random_state=1)

In [15]:
# build model

input_layer = Input(shape=(2,), name='input_layer')
x = Dense(10, activation='relu')(input_layer)
output_layer = Dense(1, name='output_layer')(x)

model = Model(inputs=input_layer, outputs = output_layer)

model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(), metrics=['accuracy'])

In [16]:
model.fit(X_train, y_train, epochs=5000, validation_data=(X_test, y_test), verbose=0)

In [17]:
model.evaluate(X_test, y_test)

1/1 [==============================] - 0s 57ms/step - loss: 7.7125


7.712474346160889

In [18]:
model.get_weights()

[array([[-0.14869678,  0.01804745, -0.58445996,  0.04870576, -0.2218143 ,
          0.52897924,  0.26451683, -0.16836232,  0.3995548 ,  0.14673102],
        [-0.01748657, -0.24496269,  0.6429859 , -0.52663517, -0.07921129,
          0.00607622,  0.07123154,  0.5905184 , -0.26153147, -0.22545952]],
       dtype=float32),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32),
 array([[ 0.56428546],
        [-0.22995114],
        [-0.68051046],
        [ 0.00238961],
        [ 0.3480007 ],
        [-0.49226105],
        [-0.36937255],
        [ 0.34868306],
        [ 0.20153987],
        [-0.03575331]], dtype=float32),
 array([0.], dtype=float32)]

In [29]:
# build sequential model

model2 = tf.keras.models.Sequential([
    Dense(1, input_shape=(2,), kernel_initializer='ones', bias_initializer='zeros',activation='sigmoid')
])

model2.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(), metrics=['accuracy'])

In [30]:
model2.fit(X_train, y_train, epochs=5000, validation_data=(X_test, y_test), verbose=0)

In [32]:
model2.evaluate(X_test, y_test)

1/1 [==============================] - 0s 31ms/step - loss: 0.4293 - accuracy: 0.8333


[0.42927733063697815, 0.8333333134651184]

In [33]:
model2.predict(X_test)

1/1 [==============================] - 0s 51ms/step


array([[0.75132936],
       [0.83543164],
       [0.27489728],
       [0.31532824],
       [0.7390143 ],
       [0.66953367]], dtype=float32)

In [34]:
y_test

24    1
17    1
19    0
20    0
14    1
3     0
Name: bought_insurance, dtype: int64

In [35]:
coef, intercept = model2.get_weights()

In [38]:
def sigmoid(X):
  import math
  return 1/ (1 + math.exp(-X))


sigmoid(18)

0.9999999847700205

In [39]:
def prediction_function(age, affordability):
  weighted_sum = coef[0] * age + coef[1] * affordability +intercept

  return sigmoid(weighted_sum)

In [40]:
prediction_function(22,1)

0.3294967688298661

In [42]:
def sigmoid_numpy(X):
  import numpy as np
  return 1/(1+np.exp(-X))

sigmoid_numpy(np.array([10,3,2]))

array([0.9999546 , 0.95257413, 0.88079708])

In [51]:
def log_loss(y_true, y_predicted):
  epsilon = 1e-15
  y_predicted_new = [max(epsilon,i) for i in y_predicted]
  y_predicted_new = [min(i, (1-epsilon)) for i in y_predicted_new]
  y_predicted_new = np.array(y_predicted)
  return -np.mean(y_true*np.log(y_predicted_new) + (1-y_true) * np.log(1-y_predicted_new))

In [64]:
def gradient_descent(age, affordability, y_true, epochs, loss_thresold):
  w1 = w2 = 1
  bias = 0
  learning_rate = 0.5
  n = len(age)

  for i in range(epochs):
    weighted_sum = age*w1 + affordability*w2 + bias
    y_predicted = sigmoid_numpy(weighted_sum)
    loss  = log_loss(y_true, y_predicted)

    w1d = (1/n)*np.dot(np.transpose(age), (y_predicted-y_true))
    w2d = (1/n)*np.dot(np.transpose(affordability), (y_predicted-y_true))
    bias_d = np.mean(y_predicted-y_true)

    w1 = w1- learning_rate * w1d
    w2 = w2- learning_rate * w2d
    bias = bias - learning_rate * bias_d
    if loss<=loss_thresold:
      break

    return w1, w2, bias


In [65]:
gradient_descent(X_train['age'],X_train['affordibility'],y_train,1000, 0.4631)

<ipython-input-51-a5da8e329434>:6: RuntimeWarning: divide by zero encountered in log
  return -np.mean(y_true*np.log(y_predicted_new) + (1-y_true) * np.log(1-y_predicted_new))


(-6.227272722469957, 0.8863636364941687, -0.24999999974133555)

In [66]:
coef, intercept

(array([[0.06486347],
        [0.5293654 ]], dtype=float32),
 array([-2.6668236], dtype=float32))